In [ ]:
import agentpy as ap
import numpy as np
import random as rd
import matplotlib.pyplot as plt
import pandas as pd

class Habitacion:
    def __init__(self, rows, columns, dirt_percentage):
        self.rows = rows
        self.columns = columns
        self.matriz = np.zeros((rows, columns), dtype=int)
        self.initialize_dirt(dirt_percentage)

    #inicializa las celdas sucias representadas con un 1
    def initialize_dirt(self, dirt_percentage):
        is_dirty = 0
        total_cells = self.rows * self.columns
        dirt_cells = int((total_cells * dirt_percentage) / 100)
        while is_dirty < dirt_cells:
            row = rd.randint(0, self.rows - 1)
            column = rd.randint(0, self.columns - 1)
            if self.matriz[row][column] == 0:
                self.matriz[row][column] = 1
                is_dirty += 1
        return self.matriz

import agentpy as ap
import numpy as np
import random as rd
import matplotlib.pyplot as plt
import pandas as pd

class Habitacion:
    def __init__(self, rows, columns, dirt_percentage):
        self.rows = rows
        self.columns = columns
        self.matriz = np.zeros((rows, columns), dtype=int)
        self.initialize_dirt(dirt_percentage)

    #inicializa las celdas sucias representadas con un 1
    def initialize_dirt(self, dirt_percentage):
        is_dirty = 0
        total_cells = self.rows * self.columns
        dirt_cells = int((total_cells * dirt_percentage) / 100)
        while is_dirty < dirt_cells:
            row = rd.randint(0, self.rows - 1)
            column = rd.randint(0, self.columns - 1)
            if self.matriz[row][column] == 0:
                self.matriz[row][column] = 1
                is_dirty += 1
        return self.matriz


class cleaningAgent(ap.Agent):
    def setup(self):
      #inicializa a los agentes en la posicion [1,1]
        self.pos = (1,1)
        self.utilidad = 0
        self.moves=0
        self.memory = set()
        self.time_step = 0 

    def see(self):
        #Devuelve una lista de posiciones válidas (dentro de los límites) 
        #adyacentes a la posición actual.
        #Se permiten movimientos en 8 direcciones.
        row,col=self.pos
        neighbors=[
            (row-1,col), #arriba
            (row+1,col), #abajo
            (row,col-1), #izquierda
            (row,col+1), #derecha
            (row-1,col-1), #arriba-izquierda
            (row-1,col+1), #arriba-derecha
            (row+1,col-1), #abajo-izquierda
            (row+1,col+1) #abajo-derecha
        ]
        valid_neighbors=[]
        for r, c in neighbors:
          if (0 <= r < self.model.habitacion.rows 
              and 0 <= c < self.model.habitacion.columns):
            valid_neighbors.append((r, c))
        return valid_neighbors

    #verifica si la celda actual esta sucia
    def is_dirty(self):
      row,col=self.pos
      return self.model.habitacion.matriz[row][col] == 1


    #funcion action()  
    def clean(self):
      row,col=self.pos
      #limpia la celda si esta sucia  
      if self.is_dirty():
        self.model.habitacion.matriz[row][col]=0 
        self.utilidad+=1
        self.memory.add((row,col))

     
    #funcion de toma de decisiones
    def next_action(self):
        current_pos=self.pos
        #decision 1: si la celda esta sucia, la limpia
        if self.is_dirty():
          self.clean()
         #decision 2: mueve el agente a una celda aleatoria adyacente. 
        else:
          neighbors=self.see()
          if neighbors:
            next_pos=rd.choice(neighbors)
            self.pos = next_pos
        self.time_step+=1

class CleaningAgent(ap.Model):

    def setup(self):
      #crea una matriz de habaitacion de 10x8 con 75% de suciedad
      self.habitacion=Habitacion(10,8,75)
      self.agents=ap.AgentList(self,self.p.agents,cleaningAgent)
      self.total_steps=0

    def step(self):
        self.agents.next_action() #ejecuta el metodo para todos los agentes
        self.clean_cells=np.sum(self.habitacion.matriz==0)
        self.total_steps += 1 
        #finaliza la corrida si todas las celdas estan limpias
        if self.clean_cells==(self.habitacion.rows * self.habitacion.columns):
          self.end()
          self.stop()
        elif self.total_steps>=self.p.max_steps:
          self.end()
          self.stop()
        

    def update(self):
        self.record('total_steps',self.total_steps)


    def end(self):
      total_cells = self.habitacion.rows * self.habitacion.columns
      remaining_dirty_cells = np.sum(self.habitacion.matriz == 1)
      clean_cells_percentage=(np.sum(self.habitacion.matriz==0)/total_cells)*100
      self.record('remaining_dirty_cells', remaining_dirty_cells)
      self.record('clean_cells_percentage', clean_cells_percentage)
      self.record('utilidad', sum(a.utilidad for a in self.agents))
      self.record('total_steps', self.total_steps)

#funcion para recopilar resultados y ejecutar simulacion
def run(percentage):
  parameters={
    'agents':10,
    'max_steps':222*percentage
    }
  return parameters 

simula = []
for percentage in [0.25, 0.50, 0.75, 1.0]:
  model = CleaningAgent(run(percentage))
  results = model.run()
  cleaning_agent_df = results.variables['CleaningAgent']
  # Obtener los últimos valores de cada variable
  simula.append({
      'run_time_percentage': percentage * 100,
      'remaining_dirty_cells':cleaning_agent_df['remaining_dirty_cells'].iloc[-1],
      'clean_cells_percentage':cleaning_agent_df['clean_cells_percentage'].iloc[-1],
      'utilidad': cleaning_agent_df['utilidad'].iloc[-1],
      'total_steps': cleaning_agent_df['total_steps'].iloc[-1]
    })

#extrae los valores de cada métrica para graficarlos
percentages = [s['run_time_percentage'] for s in simula]
remaining_dirty_cells = [s['remaining_dirty_cells'] for s in simula]
clean_cells_percentage = [s['clean_cells_percentage'] for s in simula]
utilidad = [s['utilidad'] for s in simula]
total_steps = [s['total_steps'] for s in simula]

# Crea una figura y un conjunto de subgráficos
fig, axs = plt.subplots(2, 2, figsize=(10, 10))

# Grafica 'remaining_dirty_cells'
axs[0, 0].plot(percentages, remaining_dirty_cells, marker='o', color='r')
axs[0, 0].set_title('Remaining Dirty Cells')
axs[0, 0].set_xlabel('Run Time Percentage')
axs[0, 0].set_ylabel('Remaining Dirty Cells')

# Grafica 'clean_cells_percentage'
axs[0, 1].plot(percentages, clean_cells_percentage, marker='o', color='g')
axs[0, 1].set_title('Clean Cells Percentage')
axs[0, 1].set_xlabel('Run Time Percentage')
axs[0, 1].set_ylabel('Clean Cells Percentage')

# Grafica 'utilidad'
axs[1, 0].plot(percentages, utilidad, marker='o', color='b')
axs[1, 0].set_title('Utilidad')
axs[1, 0].set_xlabel('Run Time Percentage')
axs[1, 0].set_ylabel('Utilidad')

# Grafica 'total_steps'
axs[1, 1].plot(percentages, total_steps, marker='o', color='orange')
axs[1, 1].set_title('Total Steps')
axs[1, 1].set_xlabel('Run Time Percentage')
axs[1, 1].set_ylabel('Total Steps')


plt.tight_layout()
plt.show()

#agente optimo

#función para calcular la puntuación del agente
def calcular_puntuacion(agent_data, max_steps, total_cells):
    utilidad = agent_data['utilidad']
    clean_cells_percentage = agent_data['clean_cells_percentage']
    total_steps = agent_data['total_steps']
    remaining_dirty_cells = agent_data['remaining_dirty_cells']
    # Ponderaciones para cada métrica
    peso_utilidad = 0.2
    peso_limpieza = 0.3
    peso_steps = 0.4
    peso_suciedad = 0.1
    # Calcula la puntuación para cada métrica
    puntuacion = (
        (utilidad / max_utilidad) * peso_utilidad + 
        (clean_cells_percentage / 100) * peso_limpieza + 
        ((1 - (total_steps / max_steps)) * peso_steps) + 
        ((1 - (remaining_dirty_cells / total_cells)) * peso_suciedad)
    )
    return puntuacion

# Encuentra el agente óptimo al comparar las puntuaciones
max_utilidad = max([s['utilidad'] for s in simula]) 
max_steps = max([s['total_steps'] for s in simula]) 
total_cells = 10 * 8  # Número total de celdas

# Calcular la puntuación de cada agente
simula_puntuadas = []
for agent_data in simula:
    puntuacion = calcular_puntuacion(agent_data, max_steps, total_cells)
    agent_data['puntuacion'] = puntuacion
    simula_puntuadas.append(agent_data)

# Ordenar los agentes según su puntuación (de mayor a menor)
simula_puntuadas.sort(key=lambda x: x['puntuacion'], reverse=True)

# Mostrar al agente óptimo
agente_optimo = simula_puntuadas[0]
print(f"El agente óptimo tiene una puntuación de {agente_optimo['puntuacion']:.4f}")
print(f"Detalles del agente óptimo: {agente_optimo}")




        